## 实例1 从面向过程到面向函数

- 某个飞镖盘上有7个得分区域，分值分别为3, 7, 11, 14, 18, 20和25。
- 现有3个飞镖，在都不落空的前提下，能获得32分的打法有哪几种？

(答案：三种，[3, 11, 18]，[7, 7, 18]和[7, 11, 14]）


### 传统解法

面向过程的传统解法一般基于回溯和递归来实现

In [6]:
def solve(zones, darts, target, offset, hits, answers):
  if darts == 0:
    if sum(hits) == target:
      answers.append(list(hits))
    return
  
  for idx in range(offset, len(zones)):
    solve(
        zones,
        darts-1,
        target,
        idx,
        hits + [zones[idx]],
        answers
    )


answer = list()
solve(
  [3, 7, 11, 14, 18, 20],
  3,
  32,
  0,
  [],
  answer
)
print(answer)

[[3, 11, 18], [7, 7, 18], [7, 11, 14]]


### 函数编程

函数编程将问题分解成两个部分：
1. 核心问题：输入三个分数，判断这三个数是不是一个满足条件的解；
2. 外围问题：生成所有可能的分数组合

接下来将两个部分组合在一起即可——过滤出所有满足“核心问题”的分数组合。

In [9]:
from itertools import combinations_with_replacement as cwr

def is_a_solution(hits):
  return sum(hits) == 32

list(
  filter(
    is_a_solution,
    cwr(
      [3, 7, 11, 14, 18, 20],
      3
    )
  )
)

[(3, 11, 18), (7, 7, 18), (7, 11, 14)]

In [10]:
print(list(
  cwr(
    [3, 7, 11, 14, 18, 20],
    3
  ))
)

[(3, 3, 3), (3, 3, 7), (3, 3, 11), (3, 3, 14), (3, 3, 18), (3, 3, 20), (3, 7, 7), (3, 7, 11), (3, 7, 14), (3, 7, 18), (3, 7, 20), (3, 11, 11), (3, 11, 14), (3, 11, 18), (3, 11, 20), (3, 14, 14), (3, 14, 18), (3, 14, 20), (3, 18, 18), (3, 18, 20), (3, 20, 20), (7, 7, 7), (7, 7, 11), (7, 7, 14), (7, 7, 18), (7, 7, 20), (7, 11, 11), (7, 11, 14), (7, 11, 18), (7, 11, 20), (7, 14, 14), (7, 14, 18), (7, 14, 20), (7, 18, 18), (7, 18, 20), (7, 20, 20), (11, 11, 11), (11, 11, 14), (11, 11, 18), (11, 11, 20), (11, 14, 14), (11, 14, 18), (11, 14, 20), (11, 18, 18), (11, 18, 20), (11, 20, 20), (14, 14, 14), (14, 14, 18), (14, 14, 20), (14, 18, 18), (14, 18, 20), (14, 20, 20), (18, 18, 18), (18, 18, 20), (18, 20, 20), (20, 20, 20)]


## 实例2 从面向对象到面向函数

某sdk包请求http API以获得数据，但请求过程中需要用户提供一些参数名和参数值。

HTTP API：`https://httpbin.org/get?key=value` 其中key和value需要由调用者决定。



### 面向对象的封装

SDK实现虚类，调用者派生并实现提供key和value的方法

In [17]:
from requests import Session, Request

class AbstractServiceClient:
  def execute(self):
    k, v = self.get_key_value()

    req = Request(
      method='GET',
      url='https://httpbin.org/get',
      params=dict([(k, v)])
    )
    with Session() as session:
      with session.send(req.prepare()) as resp:
        print(resp.json())
  
  def get_key_value(self):
    raise NotImplementedError('should be implemented by subclasses')


class BookServiceClient(AbstractServiceClient):
  def get_key_value(self):
    return ('book_id', 'z-0102')


class MovieServiceClient(AbstractServiceClient):
  def get_key_value(self):
    return ('movie_id', 'mov-0102')


BookServiceClient().execute()
MovieServiceClient().execute()

{'args': {'book_id': 'z-0102'}, 'headers': {'Accept-Encoding': 'identity', 'Host': 'httpbin.org', 'X-Amzn-Trace-Id': 'Root=1-61c946e5-74d3d42c00dc99db0beca417'}, 'origin': '34.82.242.212', 'url': 'https://httpbin.org/get?book_id=z-0102'}
{'args': {'movie_id': 'mov-0102'}, 'headers': {'Accept-Encoding': 'identity', 'Host': 'httpbin.org', 'X-Amzn-Trace-Id': 'Root=1-61c946e5-39d8f2d2303d1f476e0e91a8'}, 'origin': '34.82.242.212', 'url': 'https://httpbin.org/get?movie_id=mov-0102'}


### 面向函数的封装

SDK将整个request传递给调用者，由调用者对request进行全面修改。

In [18]:
from requests import Session, Request


def call_api(modifier):
  req = Request(
    method='GET',
    url='https://httpbin.org/get',
    params=dict()
  )
  modifier(req)
  with Session() as session:
    with session.send(req.prepare()) as resp:
      print(resp.json())


def add_book_params(req: Request):
  req.params['book_id'] = 'z-0102'


def add_movie_params(req: Request):
  req.params['movie_id'] = 'mov-0102'


call_api(add_book_params)
call_api(add_movie_params)


{'args': {'book_id': 'z-0102'}, 'headers': {'Accept-Encoding': 'identity', 'Host': 'httpbin.org', 'X-Amzn-Trace-Id': 'Root=1-61c947d6-4f4c5b7a644a9a69104737ec'}, 'origin': '34.82.242.212', 'url': 'https://httpbin.org/get?book_id=z-0102'}
{'args': {'movie_id': 'mov-0102'}, 'headers': {'Accept-Encoding': 'identity', 'Host': 'httpbin.org', 'X-Amzn-Trace-Id': 'Root=1-61c947d7-531a3baf569e022324d7ca98'}, 'origin': '34.82.242.212', 'url': 'https://httpbin.org/get?movie_id=mov-0102'}
